In [1]:
from sklearn import preprocessing
import numpy as np
from math import sin

In [2]:
# drelu_3 = (self.ReLU(self.outputs3, deriv=True)[0] * loss_deriv).sum(axis=1, keepdims=True)
#         dinputs3 = drelu_3 * self.weights3.T # For each input to the neuron, show how the output changes
#         dweights3 = (drelu_3 * self.inputs.T).sum(axis=1, keepdims=True)
#         dbiases3 = drelu_3
        
#         self.weights3 -= (self.learning_rate * dweights3).T
#         self.biases3 -= (self.learning_rate * dbiases3).T
#         # ======================================
#         drelu_2 = (self.ReLU(self.outputs2, deriv=True)[0] * dinputs3).sum(axis=1, keepdims=True)
#         dinputs2 = drelu_2 * self.weights2.T # For each input to the neuron, show how the output changes
#         dweights2 = (drelu_2 * self.inputs.T).sum(axis=1, keepdims=True)
#         dbiases2 = drelu_2
        
#         self.weights2 -= (self.learning_rate * dweights2).T
#         self.biases2 -= (self.learning_rate * dbiases2).T
#         # =====================================
#         drelu_1 = (self.ReLU(self.outputs1, deriv=True)[0] * dinputs2).sum(axis=1, keepdims=True)
#         dinputs1 = drelu_1 * self.weights1.T # For each input to the neuron, show how the output changes
#         dweights1 = (drelu_1 * self.inputs.T).sum(axis=1, keepdims=True)
#         dbiases1 = drelu_1
        
#         self.weights1 -= (self.learning_rate * dweights1).T
#         self.biases1 -= (self.learning_rate * dbiases1).T

In [3]:
inputs = np.array([
    *[[num] for num in range (10000)]
])

targets = np.array([
    *[[sin(num)] for num in range (10000)]
])

inputs_scaler = preprocessing.StandardScaler().fit(inputs)
inputs = inputs_scaler.transform(inputs)

targets_scaler = preprocessing.StandardScaler().fit(targets)
targets = targets_scaler.transform(targets)

In [4]:
class NeuralNetwork(object):
    """
    2 neuron in the first layer with 1 input each
    2 neurons in the second layer with 2 input each
    1 neuron in the third layer with 2 inputs
    """
    
    def __init__(self):
        self.weights1 = np.random.rand(1, 2)
        self.biases1 = np.random.rand(1, 2)
    
        self.weights2 = np.random.rand(2, 2)
        self.biases2 = np.random.rand(1, 2)
    
        self.weights3 = np.random.rand(1, 2)
        self.biases3 = np.random.rand(1, 1)
        
        self.learning_rate = 0.00001
    
    @staticmethod
    def ReLU(x, deriv=False):
        if deriv:
            r = np.zeros_like(x)
            r[x > 0] = 1
            return r
        
        return np.maximum(x, 0)
    
    @staticmethod
    def MSE(predicted, targets, mean=True):
        errors = np.sum( np.power( targets - predicted, 2) )/len(targets)
        return errors.mean() if mean else errors
    
    @staticmethod
    def L2Loss(predicted, targets, deriv=False):
        if deriv:
            return 2 * np.sum(predicted, axis=1, keepdims=True)
        return np.sum( np.power(targets - predicted, 2) )
    
    def predict(self, inputs):
        self.outputs1 = inputs @ self.weights1.T + self.biases1
        self.values1 = self.ReLU(self.outputs1)
        
        self.outputs2 = self.values1 @ self.weights2.T + self.biases2
        self.values2 = self.ReLU(self.outputs2)
        
        self.outputs3 = self.values2 @ self.weights3.T + self.biases3
        self.values3 = self.ReLU(self.outputs3)
        return self.values3
    
    def forward(self, inputs, targets, silent=False):
        self.targets = targets
        self.inputs = inputs
        
        self.predict(inputs)
        
        mse = self.MSE(self.values3, targets)
        loss = self.L2Loss(self.values3, targets)
        
        if not silent:
            print(f'Mean Squared Error: {mse:.3f}')
            print(f'L2 Loss: {loss:3f}')
        
    def backwards(self):
        # For each sample (column), show how each neuron/output (row) changes the loss function value 
        loss_deriv = self.L2Loss(self.values3, self.targets, deriv=True).T # neurons x samples
        
        # ReLU deriv of 3 row shows how the ReLU function (output) of the neuron is affected by its inputs
        # ReLU input is the sum of the products for each sample (column)
        # This is the measure of how each neuron changes the loss ???
        drelu_3 = (self.ReLU(self.outputs3, deriv=True)[0] * loss_deriv).sum(axis=1, keepdims=True)
        dinputs3 = drelu_3 * self.weights3.T # For each input to the neuron, show how the output changes
        dweights3 = (drelu_3 * self.inputs.T).sum(axis=1, keepdims=True)
        dbiases3 = drelu_3
        
        self.weights3 -= (self.learning_rate * dweights3).T
        self.biases3 -= (self.learning_rate * dbiases3).T
        # ======================================
        drelu_2 = (self.ReLU(self.outputs2, deriv=True)[0] * dinputs3).sum(axis=1, keepdims=True)
        dinputs2 = drelu_2 * self.weights2.T # For each input to the neuron, show how the output changes
        dweights2 = (drelu_2 * self.inputs.T).sum(axis=1, keepdims=True)
        dbiases2 = drelu_2
        
        self.weights2 -= (self.learning_rate * dweights2).T
        self.biases2 -= (self.learning_rate * dbiases2).T
        # =====================================
        drelu_1 = (self.ReLU(self.outputs1, deriv=True)[0] * dinputs2).sum(axis=1, keepdims=True)
        dinputs1 = drelu_1 * self.weights1.T # For each input to the neuron, show how the output changes
        dweights1 = (drelu_1 * self.inputs.T).sum(axis=1, keepdims=True)
        dbiases1 = drelu_1
        
        self.weights1 -= (self.learning_rate * dweights1).T
        self.biases1 -= (self.learning_rate * dbiases1).T

In [5]:
network = NeuralNetwork()
print('Before fitting:', network.predict([[1,5]]))

for inp, answer in zip(inputs, targets):
    network.forward(np.array(inp), np.array([answer]), silent=True)
    network.backwards()
    
print('After fitting:', network.predict([[1,5]]))

Before fitting: [[3.33426474]]


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 2 is different from 1)